In [259]:
import pandas as pd
import re

In [264]:
df = pd.read_excel("Northeastern University Street Addresses.xlsx")

C:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


##### Northeastern had a completely different data format and column names

In [265]:
df.columns

Index(['Address', 'Neighborhood', 'Zipcode', 'Level', 'Time Status', 'Year',
       'Professional School', 'Home or Private?'],
      dtype='object')

In [266]:
temp = df.copy(deep=True)

In [267]:
temp.head()

,Address,Neighborhood,Zipcode,Level,Time Status,Year,Professional School,Home or Private?
0,"6 Oswald St., 2",Fenway,2115,Undergraduate,Full Time,Fall 2017,,Private
1,"146 Smith St, #5",Mission Hill,2120,Undergraduate,Full Time,Fall 2017,,Private
2,"998 Tremont Street, Apt. 1",Mission Hill,2120,Undergraduate,Full Time,Fall 2017,,Private
3,"26 Symphony Rd, Apt #3",Fenway,2115,Undergraduate,Full Time,Fall 2017,,Private
4,114 Hemenway St #3,Fenway,2115,Undergraduate,Full Time,Fall 2017,,Private


In [268]:
def remove_multiple_spaces(text):
  return re.sub(' +', ' ', text)

In [269]:
def parse_address(address): # There were many inconsistencies in the addresses provided by northeaseter hence many other cases had be considered as well
    address = str(address)
    address = address.replace(","," ")
    address = address.replace("."," ")
    address = remove_multiple_spaces(address).upper()
    # print(address)
    add_split = address.split()
    try:
        stno=int(add_split[0])
        add_split.pop(0)
    except:
        # add_split=["N/A stno"]+add_split
        stno=None
    y= set(add_split).intersection(set(['AVE',"AVENUE",'BLVD','BOULEVARD','RD',
                                        'ROAD','RD.','STEET',"ST.","ST",'STREET','DRIVE','DR','DR.']))
    if len(y)==1:
        suff = list(y)[0]
        add_split.remove(suff)
        # i=add_split.index(list(y)[0])
        # j=add_split.pop(i)
        # suff = j
    else:
        suff=None
    
    if "UNIT" in add_split:
        i=add_split.index("UNIT")
        apt = " ".join(add_split[i:])
        add_split = add_split[:i]
    elif "#" in add_split:
        i=add_split.index("#")
        apt = " ".join(add_split[i:])
        add_split = add_split[:i]
    elif "APT" in add_split:
        i=add_split.index("APT")
        apt = " ".join(add_split[i:])
        add_split = add_split[:i]
    elif "APT." in add_split:
        i=add_split.index("APT")
        apt = " ".join(add_split[i:])
        add_split = add_split[:i]
    elif "APT.." in add_split:
        i=add_split.index("APT..")
        apt = " ".join(add_split[i:])
        add_split = add_split[:i]
    else:
        if len(add_split)>=1:
            x=add_split[len(add_split)-1]
            if "#" in x:
                apt=x
                add_split.pop(-1)
            elif "APT" in x:
                apt=x
                add_split.pop(-1)
            else:
                try:
                    x=int(x)
                    apt=str(x)
                    add_split.pop(-1)
                except:
                    apt=None
        else:
            apt=None

    st_name = " ".join(add_split)
    print("STNO:",stno, "STNAME:",st_name,"SUFF:" ,suff,"APT:" ,apt)
    return stno, st_name, suff, apt
        
        


In [270]:
temp2 = pd.DataFrame(columns=['6a. \nStreet #', '6b. \nStreet Name', '6c. \nStreet Suffix  ','6d.\n Unit #'])

In [271]:
temp2[['6a. \nStreet #', '6b. \nStreet Name', '6c. \nStreet Suffix  ','6d.\n Unit #']] = temp['Address'].apply(parse_address).apply(pd.Series)

STNO: 6 STNAME: OSWALD SUFF: ST APT: 2
STNO: 146 STNAME: SMITH SUFF: ST APT: #5
STNO: 998 STNAME: TREMONT SUFF: STREET APT: APT 1
STNO: 26 STNAME: SYMPHONY SUFF: RD APT: APT #3
STNO: 114 STNAME: HEMENWAY SUFF: ST APT: #3
STNO: 32 STNAME: HILLSIDE SUFF: ST APT: #3
STNO: 31 STNAME: CUNARD SUFF: ST APT: APT #3
STNO: 103 STNAME: HEMENWAY APARTMENT SUFF: STREET APT: 12
STNO: 839 STNAME: PARKER SUFF: ST APT: APT 2
STNO: 23 STNAME: CHEROKEE SUFF: ST APT: None
STNO: 14 STNAME: PARKER HILL APARTMENT SUFF: AVE APT: #3
STNO: 103 STNAME: HEMENWAY SUFF: STREET APT: 8
STNO: 51 STNAME: PARK APARTMENT SUFF: DRIVE APT: 33
STNO: 68 STNAME: HILLSIDE SUFF: ST APT: #2
STNO: 44 STNAME: MOZART SUFF: STREET APT: #3
STNO: 61 STNAME: SOUTH HUNTINGTON SUFF: AVENUE APT: None
STNO: 660 STNAME: WASHINGTON 5G SUFF: STREET APT: None
STNO: 58 STNAME: WESTLAND SUFF: AVENUE APT: #11
STNO: 39 STNAME: MILLMONT SUFF: STREET APT: APT 2
STNO: 121 STNAME: STEPHEN SUFF: ST APT: 16#
STNO: 103 STNAME: HEMENWAY SUFF: ST APT: APT 

In [272]:
temp2.head()

,6a. \nStreet #,6b. \nStreet Name,6c. \nStreet Suffix,6d.\n Unit #
0,6.0,OSWALD,ST,2
1,146.0,SMITH,ST,#5
2,998.0,TREMONT,STREET,APT 1
3,26.0,SYMPHONY,RD,APT #3
4,114.0,HEMENWAY,ST,#3


In [273]:
temp2["6e. \nZip"]=temp["Zipcode"]

In [274]:
temp2

,6a. \nStreet #,6b. \nStreet Name,6c. \nStreet Suffix,6d.\n Unit #,6e. \nZip
0,6.0,OSWALD,ST,2,2115
1,146.0,SMITH,ST,#5,2120
2,998.0,TREMONT,STREET,APT 1,2120
3,26.0,SYMPHONY,RD,APT #3,2115
4,114.0,HEMENWAY,ST,#3,2115
...,...,...,...,...,...
10730,1.0,PARK LANE,None,APT 1410,2210
10731,15.0,CHILCOTT PLACE,None,#1,2130
10732,15.0,SYMPHONY,RD,APT2,2215
10733,37.0,PAUL GORE,STREET,#3,2130


In [275]:
def FT_PT(FullTime):
    FullTime = str(FullTime)
    if FullTime=="Full Time":
        return "FT"
    elif FullTime=="Part Time":
        return "PT"
    else:
        return None

In [276]:
temp2["8.\nFull-time (FT) or\nPart-time (PT)"] = temp["Time Status"].apply(FT_PT)

In [277]:
temp2.head()

,6a. \nStreet #,6b. \nStreet Name,6c. \nStreet Suffix,6d.\n Unit #,6e. \nZip,8.\nFull-time (FT) or\nPart-time (PT)
0,6.0,OSWALD,ST,2,2115,FT
1,146.0,SMITH,ST,#5,2120,FT
2,998.0,TREMONT,STREET,APT 1,2120,FT
3,26.0,SYMPHONY,RD,APT #3,2115,FT
4,114.0,HEMENWAY,ST,#3,2115,FT


In [278]:
temp["Level"].unique()

array(['Undergraduate', 'Graduate'], dtype=object)

In [279]:
def U_G(UG):
    UG= str(UG)
    if UG=='Undergraduate':
        return "U"
    elif UG=='Graduate':
        return "G"
    else:
        return None

In [280]:
temp2['7. \nUndergraduate (U) or Graduate (G)']=temp["Level"].apply(U_G)

In [281]:
temp2.head()

,6a. \nStreet #,6b. \nStreet Name,6c. \nStreet Suffix,6d.\n Unit #,6e. \nZip,8.\nFull-time (FT) or\nPart-time (PT),7. \nUndergraduate (U) or Graduate (G)
0,6.0,OSWALD,ST,2,2115,FT,U
1,146.0,SMITH,ST,#5,2120,FT,U
2,998.0,TREMONT,STREET,APT 1,2120,FT,U
3,26.0,SYMPHONY,RD,APT #3,2115,FT,U
4,114.0,HEMENWAY,ST,#3,2115,FT,U


In [283]:
temp2.to_csv("Northeastern University 2017-2018.csv",index=False)